In [33]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [34]:
batchsize=100
training_data = datasets.FashionMNIST(root="../fashion_mnist", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="../fashion_mnist", train=False, download=True, transform=transforms.ToTensor())
train_dataloader = DataLoader(training_data, batch_size=batchsize, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batchsize, shuffle=True)

# Define Hyperparameters

In [211]:
sequence_len = 28
input_size = 28
hidden_size = 128
num_layers = 1
num_classes = 10
num_epochs = 40
learning_rate = 0.25

In [212]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_class, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_class)

    def forward(self, X):
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        out, _ = self.lstm(X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return out

In [213]:
model = LSTM(input_size, hidden_size, num_classes, num_layers)
print(model)

LSTM(
  (lstm): LSTM(28, 128, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [214]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [215]:
def train(num_epochs, model, train_dataloader, loss_func, optimizer):
    total_steps = len(train_dataloader)
    for epoch in range(num_epochs):
        for batch, (images, labels) in enumerate(train_dataloader):
            images = images.reshape(-1, sequence_len, input_size)

            outputs = model(images)
            loss = loss_func(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch+1) % 100 == 0:
                print(f"Epoch: {epoch+1}; Batch {batch+1}; Loss: {loss.item():>4f}")

In [216]:
train(num_epochs, model, train_dataloader, loss_func, optimizer)

Epoch: 1; Batch 100; Loss: 1.579117
Epoch: 1; Batch 200; Loss: 1.158141
Epoch: 1; Batch 300; Loss: 0.998229
Epoch: 1; Batch 400; Loss: 0.707178
Epoch: 1; Batch 500; Loss: 0.999379
Epoch: 1; Batch 600; Loss: 0.547712
Epoch: 2; Batch 100; Loss: 0.640578
Epoch: 2; Batch 200; Loss: 0.548974
Epoch: 2; Batch 300; Loss: 0.671081
Epoch: 2; Batch 400; Loss: 0.549144
Epoch: 2; Batch 500; Loss: 0.503040
Epoch: 2; Batch 600; Loss: 0.526565
Epoch: 3; Batch 100; Loss: 0.545343
Epoch: 3; Batch 200; Loss: 0.326592
Epoch: 3; Batch 300; Loss: 0.528337
Epoch: 3; Batch 400; Loss: 0.506032
Epoch: 3; Batch 500; Loss: 0.339112
Epoch: 3; Batch 600; Loss: 0.421843
Epoch: 4; Batch 100; Loss: 0.476633
Epoch: 4; Batch 200; Loss: 0.398984
Epoch: 4; Batch 300; Loss: 0.298647
Epoch: 4; Batch 400; Loss: 0.466965
Epoch: 4; Batch 500; Loss: 0.515151
Epoch: 4; Batch 600; Loss: 0.567961
Epoch: 5; Batch 100; Loss: 0.449981
Epoch: 5; Batch 200; Loss: 0.402489
Epoch: 5; Batch 300; Loss: 0.382654
Epoch: 5; Batch 400; Loss: 0

In [222]:
test_images, test_labels = next(iter(test_dataloader))
test_labels

tensor([8, 5, 2, 0, 5, 8, 7, 1, 1, 4, 5, 7, 8, 7, 9, 5, 0, 1, 3, 3, 8, 4, 6, 3,
        9, 2, 5, 9, 7, 2, 0, 4, 4, 9, 1, 8, 1, 5, 6, 2, 5, 4, 4, 1, 3, 0, 1, 8,
        1, 7, 1, 4, 0, 1, 3, 2, 0, 9, 4, 6, 4, 3, 8, 3, 4, 1, 3, 1, 2, 5, 8, 9,
        3, 4, 7, 0, 1, 6, 7, 1, 0, 8, 6, 2, 5, 4, 5, 2, 3, 7, 1, 8, 3, 5, 0, 0,
        3, 1, 6, 8])

In [223]:
test_outputs = model(test_images.view(-1, 28, 28))

In [224]:
predicted = torch.max(test_outputs, 1)[1]
predicted

tensor([8, 5, 2, 0, 5, 6, 9, 1, 1, 4, 5, 7, 8, 7, 9, 5, 0, 1, 3, 3, 8, 4, 6, 3,
        9, 2, 5, 7, 7, 2, 0, 4, 4, 9, 1, 8, 3, 5, 4, 2, 5, 4, 4, 1, 3, 0, 1, 8,
        1, 7, 1, 6, 0, 1, 3, 2, 0, 9, 4, 2, 4, 3, 8, 3, 4, 1, 4, 1, 2, 5, 8, 9,
        3, 6, 7, 6, 1, 0, 7, 1, 0, 8, 0, 2, 5, 4, 5, 2, 3, 7, 1, 8, 3, 5, 0, 0,
        3, 1, 6, 8])

In [225]:
correct = [1 for i in range(100) if predicted[i] == test_labels[i]]

In [226]:
precentage_correct = sum(correct)/100
precentage_correct

0.88